In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from numpy import arange
from sklearn import preprocessing
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.model_selection import cross_validate
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import make_scorer, accuracy_score, f1_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from fpdf import FPDF 
from sklearn.pipeline import Pipeline
import pickle
import datetime
from datetime import datetime as dt
from datetime import timedelta
from keras.models import model_from_json

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import SGDClassifier

#from imblearn.over_sampling import SMOTE

In [2]:
%matplotlib inline
plt.style.use('fivethirtyeight')
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [3]:
df_churn = pd.read_pickle('./data/training/churn.pickle')

In [4]:
df_churn.head()

,TENURE,MONTHLY_CHARGES,CHURN_STATUS,GENDER_F,GENDER_M
0,1,29.85,0,1,0
1,34,56.95,0,0,1
2,2,53.85,1,0,1
3,45,42.30,0,0,1
4,2,70.70,1,1,0


In [5]:
df = df_churn.copy()

In [6]:
df.shape

(7043, 5)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7043 entries, 0 to 7042
Data columns (total 5 columns):
TENURE             7043 non-null int64
MONTHLY_CHARGES    7043 non-null float64
CHURN_STATUS       7043 non-null int64
GENDER_F           7043 non-null uint8
GENDER_M           7043 non-null uint8
dtypes: float64(1), int64(2), uint8(2)
memory usage: 233.8 KB


In [8]:
df.columns

Index(['TENURE', 'MONTHLY_CHARGES', 'CHURN_STATUS', 'GENDER_F', 'GENDER_M'], dtype='object')

In [9]:
df.describe()

,TENURE,MONTHLY_CHARGES,CHURN_STATUS,GENDER_F,GENDER_M
count,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000
mean,32.371149,64.761692,0.265370,0.495244,0.504756
std,24.559481,30.090047,0.441561,0.500013,0.500013
min,0.000000,18.250000,0.000000,0.000000,0.000000
25%,9.000000,35.500000,0.000000,0.000000,0.000000
50%,29.000000,70.350000,0.000000,0.000000,1.000000
75%,55.000000,89.850000,1.000000,1.000000,1.000000
max,72.000000,118.750000,1.000000,1.000000,1.000000


In [10]:
X = df.drop('CHURN_STATUS', axis=1)
y = df['CHURN_STATUS'].values

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train,test_size=0.25, random_state=0)

In [13]:
tstart = dt.now()
print("Starting ------ SDG Classifier")

pipe = Pipeline(steps=[
    ('sgd', SGDClassifier())
])

param_grid ={
    'sgd__penalty':('l2', 'elasticnet', 'none'),
    'sgd__l1_ratio':[0.15,0.10],
    'sgd__max_iter':[50,100,500],
    'sgd__early_stopping':[True]
}

model=GridSearchCV(estimator=pipe,
                         param_grid=param_grid,
                         scoring='roc_auc', 
                         n_jobs=-1,
                         pre_dispatch='2*n_jobs', 
                         cv=5, 
                         verbose=1,
                         return_train_score=False)

model.fit(X_train,y_train)

pkl_filename = "./models/sgd.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(model, file)

X_val_np = X_val.to_numpy()
predicted = model.predict(X_val_np)
#proba = model.predict_proba(X_val_np)

accuracy=accuracy_score(y_val, predicted)

CM = confusion_matrix(y_val, predicted)
(TN,FN,TP,FP) = (CM[0][0],CM[1][0],CM[1][1],CM[0][1])
FPR = FP/(FP+TN)
recall = TP/(TP+FN)
precision = TP / (TP+FP)
f1 = 2*(recall*precision)/(recall+precision)
tend = dt.now()
delta = tend-tstart


print("Best Params-")
print(model.best_params_)

print("\n")
print("Algorithm: ", 'SVC')
print("Accuracy: ",accuracy)
print("Recall: ", recall)
print("F1-support: ", f1)
print("FPR: ", FPR)
print("Runtime: ", delta)

print("Best Params-")
print(model.best_params_)

Starting ------ SDG Classifier
Fitting 5 folds for each of 18 candidates, totalling 90 fits
Best Params-
{'sgd__early_stopping': True, 'sgd__l1_ratio': 0.15, 'sgd__max_iter': 500, 'sgd__penalty': 'l2'}


Algorithm:  SVC
Accuracy:  0.71611071682044
Recall:  0.6986301369863014
F1-support:  0.5604395604395606
FPR:  0.2777777777777778
Runtime:  0:00:00.544541
Best Params-
{'sgd__early_stopping': True, 'sgd__l1_ratio': 0.15, 'sgd__max_iter': 500, 'sgd__penalty': 'l2'}
